In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(f"Running on GPU: {device_name}" if device_name else "⚠️ Not using GPU")

2025-05-18 07:33:47.576342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747553628.073512      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747553628.233605      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Running on GPU: /device:GPU:0


I0000 00:00:1747553649.522747      19 gpu_device.cc:2022] Created device /device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747553649.523974      19 gpu_device.cc:2022] Created device /device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [2]:
from tensorflow.keras.utils import image_dataset_from_directory

TRAIN_PATH = "/kaggle/input/face-occlusion/dataset_split/train"

train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(240, 240),
    batch_size=32,
    label_mode='categorical'
)

val_ds = image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(240, 240),
    batch_size=32,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ Loaded {num_classes} classes")

class_names = val_ds.class_names
num_classes = len(class_names)
print(f"✅ Loaded {num_classes} classes")

Found 197590 files belonging to 14114 classes.
Using 158072 files for training.


I0000 00:00:1747553721.934879      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747553721.935088      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 197590 files belonging to 14114 classes.
Using 39518 files for validation.
✅ Loaded 14114 classes
✅ Loaded 14114 classes


In [3]:
# Build EfficientNetB1 Model
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

base_model = EfficientNetB1(include_top=False, weights='imagenet', input_shape=(240, 240, 3))

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

base_model.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# EarlyStopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,  
    restore_best_weights=True,  
    verbose=1
)

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb1 (Functional)          │ (None, 8, 8, 1280)          │       6,575,239 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 14114)               │       3,627,298 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,530,473 (40.17 MB)

 Trainable params: 3,955,234 (15.09 MB)

 Non-trainable params: 6,575,239 (25.08 MB)

In [4]:
# Initial Training with EarlyStopping
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20


I0000 00:00:1747553818.638077      66 service.cc:148] XLA service 0x793930019520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747553818.639735      66 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747553818.639759      66 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747553821.251342      66 cuda_dnn.cc:529] Loaded cuDNN version 90300


   2/4940 ━━━━━━━━━━━━━━━━━━━━ 6:09 75ms/step - accuracy: 0.0000e+00 - loss: 9.5549   

I0000 00:00:1747553836.625724      66 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4940/4940 ━━━━━━━━━━━━━━━━━━━━ 485s 90ms/step - accuracy: 0.0192 - loss: 8.6993 - val_accuracy: 0.5241 - val_loss: 2.5147
Epoch 2/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 333s 67ms/step - accuracy: 0.4412 - loss: 2.7312 - val_accuracy: 0.8907 - val_loss: 0.4904
Epoch 3/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 333s 67ms/step - accuracy: 0.6787 - loss: 1.3486 - val_accuracy: 0.9600 - val_loss: 0.1677
Epoch 4/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 333s 67ms/step - accuracy: 0.7936 - loss: 0.8214 - val_accuracy: 0.9792 - val_loss: 0.0761
Epoch 5/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 332s 67ms/step - accuracy: 0.8528 - loss: 0.5675 - val_accuracy: 0.9886 - val_loss: 0.0428
Epoch 6/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 332s 67ms/step - accuracy: 0.8877 - loss: 0.4321 - val_accuracy: 0.9924 - val_loss: 0.0319
Epoch 7/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 332s 67ms/step - accuracy: 0.9084 - loss: 0.3528 - val_accuracy: 0.9940 - val_loss: 0.0257
Epoch 8/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 332s 67ms/step - accuracy: 0.9237 - lo

In [5]:
# Fine-tune the model
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune with EarlyStopping
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20


E0000 00:00:1747560370.526052      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747560370.670355      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747560371.159440      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747560371.299784      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747560371.598836      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

4939/4940 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.2660 - loss: 12.0821

E0000 00:00:1747561479.293778      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747561479.435025      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747561479.897235      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747561480.038053      67 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1309s 240ms/step - accuracy: 0.2661 - loss: 12.0794 - val_accuracy: 0.9800 - val_loss: 0.0651
Epoch 2/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1127s 228ms/step - accuracy: 0.8226 - loss: 0.7533 - val_accuracy: 0.9952 - val_loss: 0.0147
Epoch 3/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1128s 228ms/step - accuracy: 0.9173 - loss: 0.3134 - val_accuracy: 0.9969 - val_loss: 0.0088
Epoch 4/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1127s 228ms/step - accuracy: 0.9457 - loss: 0.1928 - val_accuracy: 0.9978 - val_loss: 0.0062
Epoch 5/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1128s 228ms/step - accuracy: 0.9595 - loss: 0.1416 - val_accuracy: 0.9977 - val_loss: 0.0055
Epoch 6/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1127s 228ms/step - accuracy: 0.9661 - loss: 0.1153 - val_accuracy: 0.9980 - val_loss: 0.0046
Epoch 7/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1127s 228ms/step - accuracy: 0.9719 - loss: 0.0947 - val_accuracy: 0.9982 - val_loss: 0.0041
Epoch 8/20
4940/4940 ━━━━━━━━━━━━━━━━━━━━ 1129s 228ms/step - accu

In [6]:
# Save model
model.save("/kaggle/working/efficientnetB1_model.h5")
print("✅ Model saved as efficientnetB1_model.h5")

✅ Model saved as efficientnetB1_model.h5
